In [1]:
import pandas as pd

# Replace with the actual path to your CSV file (uploaded to Colab)
# Make sure the file is in the correct directory and the name matches exactly
file_path = "MBTA_Bus_Ridership_by_Trip%2C_Season%2C_Route_Line%2C_and_Stop (1).csv"
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
print(df.head())


C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\3891913654.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


      season route_id route_variant  direction_id trip_start_time  \
0  Fall 2016        1         1-0-0             0        04:37:00   
1  Fall 2016        1         1-0-0             0        04:37:00   
2  Fall 2016        1         1-0-0             0        04:37:00   
3  Fall 2016        1         1-0-0             0        04:37:00   
4  Fall 2016        1         1-0-0             0        04:37:00   

   day_type_id day_type_name                      stop_name  stop_id  \
0  day_type_01       weekday   WASHINGTON ST OPP RUGGLES ST        1   
1  day_type_01       weekday       ALBANY ST OPP RANDALL ST    10003   
2  day_type_01       weekday  MASSACHUSETTS AVE @ SIDNEY ST      101   
3  day_type_01       weekday   MASSACHUSETTS AVE @ PROSPECT      102   
4  day_type_01       weekday  MASSACHUSETTS AVE @ BIGELOW S      104   

   stop_sequence  boardings  alightings  load_  sample_size  ObjectId  
0              2        0.4         0.3    7.8           13         1  
1       

Dataset is massive. I will be processing by chunks. 

In [2]:
chunk_size = 100000
result_df =df
# Process the data in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Sum the boardings per route and season in each chunk
    chunk_agg = chunk.groupby(['route_id', 'season'])['boardings'].sum().reset_index()
    
    # Concatenate the results to the final DataFrame
    result_df = pd.concat([result_df, chunk_agg])

# After concatenating all chunks, re-group and calculate the final sum
final_agg = result_df.groupby(['route_id', 'season'])['boardings'].sum().reset_index()

# Show the final aggregated boardings
print(final_agg)


C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\2732351304.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\2732351304.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\2732351304.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\2732351304.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\2732351304.py:4: DtypeWarning: Columns

     route_id     season  boardings
0           1  Fall 2016    51349.2
1           1  Fall 2017    56313.6
2           1  Fall 2018    51633.2
3           1  Fall 2019    61319.6
4           1  Fall 2020    24611.2
...       ...        ...        ...
1719      742  Fall 2017     5865.6
1720      746  Fall 2016     3843.3
1721      747  Fall 2016     2097.9
1722      749  Fall 2016    21152.0
1723       75  Fall 2016       18.5

[1724 rows x 3 columns]


In [3]:
result_df = pd.DataFrame()  # Initialize the final DataFrame

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Add a new column 'year' based on the 'season' column (assuming 'season' contains the year information)
    chunk['year'] = chunk['season'].str.extract(r'(\d{4})').astype(int)  # Extract year from 'season'
    
    # Group the data into two periods: 2016-2019 and Post 2019
    chunk['time_period'] = chunk['year'].apply(lambda x: '2016-2019' if 2016 <= x <= 2019 else 'Post 2019')
    
    # Aggregate the data by 'route_id' and 'time_period', summing the 'boardings'
    chunk_avg = chunk.groupby(['route_id', 'time_period'])['boardings'].sum().reset_index()
      
    # Concatenate the results to the final DataFrame
    result_df = pd.concat([result_df, chunk_avg])

# After concatenating all chunks, re-group and calculate the final aggregate
final_avg = result_df.groupby(['route_id', 'time_period'])['boardings'].sum().reset_index()

# Show the final results
print(final_avg)


C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\860188956.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\860188956.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\860188956.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\860188956.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\junhy\AppData\Local\Temp\ipykernel_24704\860188956.py:3: DtypeWarning: Columns (1) 

    route_id time_period  boardings
0          1   2016-2019   110307.8
1          1   Post 2019   101866.0
2          4   2016-2019     1093.3
3          4   Post 2019      427.9
4          5   2016-2019      349.6
..       ...         ...        ...
398      713   2016-2019     2275.8
399       72   2016-2019     3756.2
400       73   2016-2019    20545.8
401       74   2016-2019     2269.6
402      741   2016-2019    13409.5

[403 rows x 3 columns]


In [30]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Avoid line wrapping
pd.set_option('display.max_colwidth', None)  # Show full content of each column

# Assuming final_avg is the DataFrame you're working with
print(final_avg)

    route_id time_period      boardings
0          1   2016-2019  110307.800000
1          1   Post 2019  101866.000000
2          4   2016-2019    1093.300000
3          4   Post 2019     427.900000
4          5   2016-2019     349.600000
5          7   2016-2019    4214.300000
6          7   Post 2019   12741.000000
7          8   2016-2019   21151.100000
8          8   Post 2019   24200.800000
9          9   2016-2019   40297.500000
10         9   Post 2019   32863.300000
11        10   2016-2019   21287.800000
12        10   Post 2019   19923.000000
13        11   2016-2019   18953.400000
14        11   Post 2019   12088.000000
15        14   2016-2019    8519.300000
16        14   Post 2019    8404.900000
17        15   2016-2019   51574.100000
18        15   Post 2019   53611.000000
19        16   2016-2019   39305.500000
20        16   Post 2019   53836.900000
21        17   2016-2019   19984.800000
22        17   Post 2019   18399.200000
23        18   2016-2019    2371.800000


In [4]:
# Group by 'route_id' and 'time_period' to check if both pre and post 2019 exist for each route
route_seasons = final_avg.groupby('route_id')['time_period'].apply(set)

# Filter out routes that don't have both '2016-2019' and 'Post 2019'
valid_routes = route_seasons[route_seasons.apply(lambda x: {'2016-2019', 'Post 2019'}.issubset(x))].index

# Filter the original DataFrame to keep only valid routes
filtered_df = final_avg[final_avg['route_id'].isin(valid_routes)]

# Display the filtered DataFrame
print(filtered_df)


    route_id time_period  boardings
0          1   2016-2019   110307.8
1          1   Post 2019   101866.0
2          4   2016-2019     1093.3
3          4   Post 2019      427.9
5          7   2016-2019     4214.3
..       ...         ...        ...
355       39   Post 2019    21911.6
356        4   2016-2019      372.0
357        4   Post 2019      275.1
358       40   2016-2019      927.0
359       40   Post 2019     1179.7

[332 rows x 3 columns]


In [5]:
# Pivot the data to have '2016-2019' and 'Post 2019' as columns for each route
pivoted_df = filtered_df.pivot(index='route_id', columns='time_period', values='boardings')
# absolute change between
pivoted_df['absolute_change'] = (pivoted_df['Post 2019'] - pivoted_df['2016-2019'])
# Reset the index
pivoted_df.reset_index(inplace=True)
print(pivoted_df[['route_id', '2016-2019', 'Post 2019', 'absolute_change']])


time_period route_id  2016-2019  Post 2019  absolute_change
0                  1   110307.8   101866.0          -8441.8
1                  4     1093.3      427.9           -665.4
2                  7     4214.3    12741.0           8526.7
3                  8    21151.1    24200.8           3049.7
4                  9    40297.5    32863.3          -7434.2
..               ...        ...        ...              ...
161               37     1900.1     2315.5            415.4
162               38      983.9     1389.0            405.1
163               39    23319.0    21911.6          -1407.4
164                4      372.0      275.1            -96.9
165               40      927.0     1179.7            252.7

[166 rows x 4 columns]


In [15]:
average_percent_change = pivoted_df['absolute_change'].median()
print(f"The average percent change across all routes is: {average_percent_change:.2f}")

The average percent change across all routes is: -558.55
